In [1]:
%load_ext autoreload
%autoreload 2

# Load Data

In [2]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=25000, n_samples=50000)
dm.setup()

In [3]:
import torch
from opensynth.models.faraday import FaradayVAE
vae_model = torch.load("vae_model.pt")

/var/folders/lj/tc13qh690297rzt9q9c3qdbm0000gn/T/ipykernel_26202/3246375714.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae_model = torch.load("vae_model.pt")


In [4]:
from opensynth.models.faraday.gaussian_mixture.prepare_gmm_input import encode_data_for_gmm

next_batch = next(iter(dm.train_dataloader()))
input_tensor = encode_data_for_gmm(data=next_batch, vae_module=vae_model)
input_data = input_tensor.detach().numpy()
n_samples = len(input_tensor)

In [5]:
N_COMPONENTS = 500
REG_COVAR = 1e-4
EPOCHS = 10

# Init GMM

In [6]:
from opensynth.models.faraday.new_gmm import gmm_utils

labels_, means_, responsibilities_ = gmm_utils.initialise_centroids(
        X=input_data, n_components=N_COMPONENTS
    )
print(labels_.dtype, responsibilities_.dtype, means_.dtype)

torch.float32 torch.float32 torch.float32


In [13]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params

gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
print(gmm_init_params["precision_cholesky"][429][0][0])

tensor(0.6610)


In [14]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params, training_loop
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureModel


gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
torch_gmm = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR
)
print(f"Initial Prec Chol: {gmm_init_params['precision_cholesky'][429][0][0]}")
torch_gmm.initialise(gmm_init_params)
trained_model = training_loop(model=torch_gmm, data=input_tensor, max_iter=EPOCHS)

Initial Prec Chol: 0.6610455513000488


 10%|█         | 1/10 [00:01<00:14,  1.61s/it]

Old Prec Chol: -2.546790361404419
New Prec Chol: 0.6610455513000488


 20%|██        | 2/10 [00:03<00:11,  1.50s/it]

Old Prec Chol: 0.6610455513000488
New Prec Chol: 0.6610455513000488


...

Old Prec Chol: 0.6610455513000488
New Prec Chol: 0.6610455513000488


 90%|█████████ | 9/10 [00:13<00:01,  1.58s/it]

Old Prec Chol: 0.6610455513000488
New Prec Chol: 0.6610455513000488


100%|██████████| 10/10 [00:15<00:00,  1.56s/it]

Old Prec Chol: 0.6610455513000488
New Prec Chol: 0.6610455513000488


# SK Learn GMM Manual

In [15]:
import numpy as np
from scipy.special import logsumexp
from scipy import linalg

def _estimate_gaussian_parameters(X, resp, reg_covar=REG_COVAR):
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]
    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[:: n_features + 1] += reg_covar
    return nk, means, covariances

def _compute_precision_cholesky(covariances):
    estimate_precision_error_message = (
        "Fitting the mixture model failed because some components have "
        "ill-defined empirical covariance (for instance caused by singleton "
        "or collapsed samples). Try to decrease the number of components, "
        "or increase reg_covar."
    )

    n_components, n_features, _ = covariances.shape
    precisions_chol = np.empty((n_components, n_features, n_features))
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
        except linalg.LinAlgError:
            raise ValueError(estimate_precision_error_message)
        precisions_chol[k] = linalg.solve_triangular(
            cov_chol, np.eye(n_features), lower=True
        ).T
    return precisions_chol

def _compute_log_det_cholesky(matrix_chol, n_features):
    n_components, _, _ = matrix_chol.shape
    log_det_chol = np.sum(
        np.log(matrix_chol.reshape(n_components, -1)[:, :: n_features + 1]), 1
    )
    return log_det_chol

def _estimate_log_gaussian_prob(X, means, precisions_chol):
    n_samples, n_features = X.shape
    n_components, _ = means.shape

    log_det = _compute_log_det_cholesky(precisions_chol, n_features)

    log_prob = np.empty((n_samples, n_components))
    for k, (mu, prec_chol) in enumerate(zip(means, precisions_chol)):
        y = np.dot(X, prec_chol) - np.dot(mu, prec_chol)
        log_prob[:, k] = np.sum(np.square(y), axis=1)
    return -0.5 * (n_features * np.log(2 * np.pi) + log_prob) + log_det

def _estimate_log_weights(weights):
        return np.log(weights)

def _estimate_weighted_log_prob(X, means, precisions_chol, weights):
        return _estimate_log_gaussian_prob(X, means, precisions_chol) + _estimate_log_weights(weights)


def _estimate_log_prob_resp(X, means, precisions_chol, weights):
    weighted_log_prob = _estimate_weighted_log_prob(X, means, precisions_chol, weights)
    log_prob_norm = logsumexp(weighted_log_prob, axis=1)
    with np.errstate(under="ignore"):
        log_resp = weighted_log_prob - log_prob_norm[:, np.newaxis]
    return log_prob_norm, log_resp

def _e_step(X,means, precisions_chol, weights):
    log_prob_norm, log_resp = _estimate_log_prob_resp(X, means, precisions_chol, weights)
    return np.mean(log_prob_norm), log_resp

def _m_step(X, log_reponsibilities, reg_covar=REG_COVAR):

    weights_, means_, covariances_ = _estimate_gaussian_parameters(X,np.exp(log_reponsibilities),reg_covar=reg_covar)

    precision_cholesky_ = _compute_precision_cholesky(covariances=covariances_)

    return precision_cholesky_, weights_, means_

In [16]:
means = gmm_init_params["means"].detach().numpy()
weights = gmm_init_params["weights"].detach().numpy()
prec_chol = gmm_init_params["precision_cholesky"].detach().numpy()
print(f"Initial prec chol: {prec_chol[429][0][0]}")
for i in range(EPOCHS):
    print(f"Old Prec Chol: {prec_chol[429][0][0]}")
    log_prob, log_resp = _e_step(input_data, means, prec_chol, weights)
    prec_chol, weights, means = _m_step(input_data, log_resp)
    print(f"New prec chol: {prec_chol[429][0][0]}")

Initial prec chol: 0.6610455513000488
Old Prec Chol: 0.6610455513000488
New prec chol: 0.6610599510474371
Old Prec Chol: 0.6610599510474371
New prec chol: 0.6610599501952272
Old Prec Chol: 0.6610599501952272
New prec chol: 0.661059950782588
Old Prec Chol: 0.661059950782588
New prec chol: 0.6610599566386626
Old Prec Chol: 0.6610599566386626
New prec chol: 0.661059956929652
Old Prec Chol: 0.661059956929652
New prec chol: 0.6610599576515817
Old Prec Chol: 0.6610599576515817
New prec chol: 0.6610599577439455
Old Prec Chol: 0.6610599577439455
New prec chol: 0.6610599576759897
Old Prec Chol: 0.6610599576759897
New prec chol: 0.6610599576488254
Old Prec Chol: 0.6610599576488254
New prec chol: 0.6610599576640814


# SK Learn GMM Epoch

In [17]:
from sklearn.mixture import GaussianMixture

init_weights = gmm_init_params["weights"].detach().numpy()
init_means = gmm_init_params["means"].detach().numpy()

skgmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type='full', max_iter=EPOCHS-1, random_state=0, means_init = init_means, weights_init=init_weights)
skgmm.fit(input_data)
skgmm_pred = skgmm.predict(input_data)

/Users/shengchai/.local/share/virtualenvs/OpenSynth-EhRIPYd3/lib/python3.11/site-packages/sklearn/mixture/_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


In [18]:
IDX = 429

In [19]:
skgmm.means_[IDX]

array([-0.9553091 ,  2.27554793, -5.46096161,  2.15791613, -2.91830856,
        1.58050201,  0.77627237, -6.23789244,  3.35874238,  2.21445032,
        1.37807806, -8.67672764,  3.91632371, -2.271522  , -6.99348675,
        1.27280559,  9.        ,  3.93333333])

In [20]:
trained_model.means[IDX]

tensor([-0.9553,  2.2755, -5.4610,  2.1579, -2.9183,  1.5805,  0.7763, -6.2379,
         3.3587,  2.2145,  1.3781, -8.6767,  3.9163, -2.2715, -6.9935,  1.2728,
         9.0000,  3.9333], grad_fn=<SelectBackward0>)

In [21]:
means[IDX]

array([-0.9553091 ,  2.27554793, -5.46096162,  2.15791613, -2.91830857,
        1.58050201,  0.77627237, -6.23789244,  3.35874238,  2.21445032,
        1.37807806, -8.67672765,  3.91632371, -2.271522  , -6.99348676,
        1.27280559,  9.        ,  3.93333333])

In [22]:
gmm_init_params["means"][IDX]

tensor([-0.9553,  2.2755, -5.4610,  2.1579, -2.9183,  1.5805,  0.7763, -6.2379,
         3.3587,  2.2145,  1.3781, -8.6767,  3.9163, -2.2715, -6.9935,  1.2728,
         9.0000,  3.9333])

In [23]:
skgmm.precisions_cholesky_[IDX][0]

array([ 6.61074259e-01,  8.36924025e-01,  3.65196019e-01,  1.05916592e+00,
        8.48945403e-01,  5.61359945e+00, -3.90791031e+00, -1.54561659e+00,
        1.92295091e+00,  9.16734116e-02,  2.02745663e+00, -3.56853453e+00,
       -8.07737201e+00,  1.02246414e+01,  1.52431856e+02,  3.37481508e+02,
       -2.41489302e+01,  2.74421396e+02])

In [24]:
trained_model.precision_cholesky[IDX][0]

tensor([  0.6610,   0.8366,   0.3649,   1.0579,   0.8446,   5.4502,  -3.7044,
         -1.0665,   1.4455,   0.5109,   1.0574,  -1.1073,  -6.9742,   7.8023,
         10.5210,  15.6301, -21.1093,  20.5154])

In [25]:
prec_chol[IDX][0]

array([ 0.66105996,  0.83681011,  0.36515123,  1.05883226,  0.8480947 ,
        5.58529745, -3.86210937, -1.43949635,  1.84633907,  0.22198294,
        1.85417969, -2.52499058, -7.38099495,  8.05099365, 18.1656995 ,
       30.2713674 , -2.01883134, 28.10801538])

In [26]:
gmm_init_params["precision_cholesky"][IDX][0]

tensor([  0.6610,   0.8366,   0.3649,   1.0579,   0.8446,   5.4502,  -3.7044,
         -1.0665,   1.4455,   0.5109,   1.0574,  -1.1073,  -6.9742,   7.8023,
         10.5210,  15.6301, -21.1093,  20.5154])